In [1]:
# Import custom libaries
import classification_utils as clu
import md_utils as mdu
# Import common libraries
import time
import plotly_express as px
import os, glob

In [ ]:
# Set input variables
fnames = ['/net/jam-amaro-shared/dse_project/Spike_Dataset/TRAJECTORIES_spike_open_prot_glyc_amarolab/results/FinalExtractedFeature_open.csv',
        '/net/jam-amaro-shared/dse_project/Spike_Dataset/TRAJECTORIES_spike_closed_prot_glyc_amarolab/results/FinalExtractedFeature_closed.csv']
is_open = [1, 0]
traj_dir = '/net/jam-amaro-shared/dse_project/Spike_Dataset/TRAJECTORIES_spike_open_prot_glyc_amarolab/'

In [ ]:
# Test loading of data
# Load data
df = clu.load_data(fnames,is_open)


In [ ]:
# Test model training 
t0 = time.time()
tr_p, tr_r, ts_p, ts_r, df_feat = clu.train_sgd_model(fnames,is_open,feat_incl=['_x','_y','_z','RMSD','ROF'])
print('Train precision: ' + str(tr_p) + ', Train recall: ' + str(tr_r))
print('Test precision: ' + str(ts_p) + ', Test recall: ' + str(ts_r))
print(time.time()-t0)

In [ ]:
# Test bar chart
fig1 = clu.plot_feature_importances(df_feat)
fig1.show()

In [ ]:
# Test spike viz

# Load
t0 = time.time()
traj = mdu.load_traj(traj_dir)
t1 = time.time()
print(str(t1 - t0) + ' sec to load')

# Parse
atom_id = mdu.parse_traj(traj)
t2 = time.time()
print(str(t2-t1) + ' sec to parse')

# Viz
fig2 = mdu.viz_traj(traj,atom_id,df_feat,'Open Spike')
t3 = time.time()
print(str(t3-t2) + ' sec to viz')
fig2.show()

In [2]:
# Test parsing of data files
data_dir = '/net/jam-amaro-shared/dse_project/Spike_Dataset/'
traj_dirs = glob.glob(os.path.join(data_dir,'*TRAJECTOR*'))
traj_dirs

['/net/jam-amaro-shared/dse_project/Spike_Dataset/TRAJECTORIES_spike_mutant_prot_glyc_amarolab',
 '/net/jam-amaro-shared/dse_project/Spike_Dataset/TRAJECTORIES_spike_closed_prot_glyc_amarolab',
 '/net/jam-amaro-shared/dse_project/Spike_Dataset/TRAJECTORIES_spike_open_prot_glyc_amarolab',
 '/net/jam-amaro-shared/dse_project/Spike_Dataset/TRAJECTORIES_continuous_spike_opening_WE_chong_and_amarolab']

In [17]:
traj_sel = traj_dirs[1:3]
traj_sel

['/net/jam-amaro-shared/dse_project/Spike_Dataset/TRAJECTORIES_spike_closed_prot_glyc_amarolab',
 '/net/jam-amaro-shared/dse_project/Spike_Dataset/TRAJECTORIES_spike_open_prot_glyc_amarolab']

In [21]:
traj_sel[['closed' in i for i in traj_sel].index(True)]

'/net/jam-amaro-shared/dse_project/Spike_Dataset/TRAJECTORIES_spike_closed_prot_glyc_amarolab'

In [16]:
glob.glob(os.path.join(traj_sel,'results','*.csv'))

['/net/jam-amaro-shared/dse_project/Spike_Dataset/TRAJECTORIES_spike_closed_prot_glyc_amarolab/results/FinalExtractedFeature_1.csv',
 '/net/jam-amaro-shared/dse_project/Spike_Dataset/TRAJECTORIES_spike_closed_prot_glyc_amarolab/results/FinalExtractedFeature_3.csv',
 '/net/jam-amaro-shared/dse_project/Spike_Dataset/TRAJECTORIES_spike_closed_prot_glyc_amarolab/results/FinalExtractedFeature_2.csv',
 '/net/jam-amaro-shared/dse_project/Spike_Dataset/TRAJECTORIES_spike_closed_prot_glyc_amarolab/results/FinalExtractedFeature_closed.csv']

In [12]:
is_open = ['closed' not in d for d in traj_dirs]
is_open

[True, False, True, True]